정형 데이터: 구조화하기 쉬운 수치와 같은 데이터
비정형 데이터: 구조화하기 어려운 텍스트, 이미지, 음성과 같은 데이터

정형 데이터를 다루는 머신러닝 알고리즘의 끝판왕
--> 결정 트리 기반의 앙상블 러닝(Ensemble Learning)

비정형 데이터는 규칙성을 찾기 어려워, 전통적인 머신러닝 방법으로는 모델을 만들기 까다롭다.
--> 신경망 알고리즘(딥러닝)

이 장에서는 앙상블 학습 알고리즘에 대해 알아보자.

# 랜덤 포레스트

랜덤 포레스트는
- 결정 트리를 랜덤하게 만들어 '결정 트리의 숲'을 만든다.
- 각 결정 트리의 예측을 사용해 최종 예측을 만든다.

먼제 랜덤 포레스트는 각 트리를 훈련하기 위한 데이터를 랜덤하게 만드는데, 이 방식이 독특하다.
- 우리가 입력한 훈련 데이터에서 랜덤하게 샘플을 추출하여 훈련 데이터를 만든다.
    - 이 때, 한 샘플이 중복되어 추출될 수 있다. --> 부트스트랩 샘플
        - 부트스트랩: 데이터 세트에서 중복을 허용하여 데이터를 샘플링하는 방식
    - 기본적으로 부트스트랩 샘플은 훈련 세트의 크기와 같게 만든다.
- 또한, 각 노드를 분할할 때, 전체 특성 중에서 일부 특성을 무작위로 고른 다음 이 중에서 최선의 분할을 찾는다.
    - RandomForestClassifier는 기본적으로 전체 특성 개수의 제곱근만큼의 특성을 고른다.
        - 4개의 특성이면 노드마다 랜덤하게 2개를 고르고, 
        - 그 2개의 특성에서 최선의 분할을 선택한다.
    - 단, 회귀 모델인 RandomForestRegressor는 전체 특성을 모두 사용한다.

사이킷런의 랜덤 포레스트는 기본적으로 100개의 결정 트리를 이런 방식으로 훈련한다.
- 그 후, 분류일 때는, 각 트리의 클래스별 확률을 평균하여 가장 높은 확률을 가진 클래스를 예측으로 삼는다.
- 회귀일 때는, 각 트리의 예측을 평균한다.


---

랜덤 포레스트는 랜덤하게 선택한 샘플과 특성을 사용하기 때문에, 과대적합을 막아주고 검증 세트와 테스트 세트에서 안정적인 성능을 얻을 수 있다.

그럼 와인 분류 문제에 랜덤 포레스트 분류를 적용해보자.

In [1]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split

wine = pd.read_csv('https://bit.ly/wine_csv_data')
data = wine[['alcohol', 'sugar', 'pH']].to_numpy()
target = wine['class'].to_numpy()

train_input, test_input, train_target, test_target = train_test_split(data, target, test_size=0.2, random_state=42)

랜덤 포레스트 객체를 만들고, 교차 검증을 수행해보자.

기본적으로 100개의 트리를 만들어내기 때문에, n_jobs=-1로 두자.

In [3]:
from sklearn.model_selection import cross_validate
from sklearn.ensemble import RandomForestClassifier

rf = RandomForestClassifier(n_jobs=-1, random_state=42)
scores = cross_validate(rf, train_input, train_target, return_train_score=True, n_jobs=-1) # 훈련 평가 점수도 확인

print(np.mean(scores['train_score']), np.mean(scores['test_score']))

0.9973541965122431 0.8905151032797809


다소 과대적합이 발생한 것 같다.

랜덤 포레스트는 결정 트리의 앙상블이기 때문에, DecisionTreeClassifier가 제공하는 주요 매개변수를 모두 지원한다.
- criterion, max_depth, max_features, min_sample_split, min_impurity_decrease, min_sample_leaf 등
- 또한 특성 중요도도 계산할 수 있다
    - 이는 각 결정 트리의 특성 중요도를 취합한 값이다.

앞서 만든 랜덤 포레스트 모델을 훈련 세트에 훈련시킨 후 특성 중요도를 출력해보자.

In [5]:
rf.fit(train_input, train_target)
print(rf.feature_importances_)

[0.23167441 0.50039841 0.26792718]


단일 결정 트리와는 다르게, 알코올 도수와 pH의 특성 중요도가 조금 상승했다.
- 이는, 랜덤 포레스트 분류는, 특성의 일부를 선택하여 결정 트리를 훈련시키기 때문
- 그 결과 좀 더 많은 특성이 훈련에 기여할 기회를 얻는다.
- --> 과대적합을 줄이고 일반화 성능을 높이는 데 도움이 된다.

---

RandomForestClassifier은 자체적으로 모델을 평가하는 점수를 얻을 수 있다.
- OOB(out of bag) 샘플: 결정 트리를 훈련할 때 구성한 부트스트랩 샘플에 포함되지 않고 남은 샘플
- OOB 샘플을 이용하여, 부트스트랩 샘플로 훈련한 결정 트리를 평가할 수 있다. (마치 검증 세트처럼)
- 이 점수를 얻으려면 oob_score 매개변수를 True로 지정해야 한다. (default: False)

In [6]:
rf = RandomForestClassifier(oob_score=True, n_jobs=-1, random_state=42)
rf.fit(train_input, train_target)
print(rf.oob_score_)

0.8934000384837406


교차 검증으로 얻은 점수와 비슷한 점수가 나왔다!

OOB 점수를 사용하면 교차 검증을 대신할 수 있어서, 결과적으로 훈련 세트에 더 많은 샘플을 사용할 수 있다.

## 엑스트라 트리

랜덤 포레스트와 매우 비슷하게 동작한다.
- 기본적으로 100개의 트리를 훈련하고, 대부분의 매개변수를 공유하고, 일부 특성을 선택하여 노드를 분할하는 데 사용한다.

그러나 랜덤 포레스트와의 차이는,
- 부트스트랩 샘플을 사용하지 않는다. --> 결정 트리를 만들 때 전체 훈련 세트를 사용한다.
- 대신 노드를 분할할 때, 가장 좋은 분할을 찾지 않고, 무작위로 분할한다.
    - --> DecisionTreeClassifier(splitter='random')인 결정 트리를 이용한다.
- 하나의 결정 트리에서 랜덤하네 노드를 분할하면 성능이 낮아지겠지만(05-2 학습문제 3에서 봤듯이)
- 많은 트리를 앙상블 하기 때문에, 과대적합을 막고 검증 세트의 점수를 높이는 효과가 있다.

In [7]:
from sklearn.ensemble import ExtraTreesClassifier

et = ExtraTreesClassifier(n_jobs=-1, random_state=42)
scores = cross_validate(et, train_input, train_target, return_train_score=True, n_jobs=-1)

print(np.mean(scores['train_score']), np.mean(scores['test_score']))

0.9974503966084433 0.8887848893166506


랜덤 포레스트와 비슷한 결과를 얻었다.
- 이 예제는 특성이 많지 않아 두 모델의 차이가 크지 않다.
- 보통 엑스트라 트리가 무작위성이 크기 때문에 더 많은 결정 트리를 훈련해야 한다.
- 하지만, 랜덤하게 노드를 분할하기 때문에 계산속도가 빠르다는 것이 장점이다.
    - 결정 트리는 최적의 분할을 찾는 데 시간을 많이 소모한다. (특히 특성이 많을 때)

엑스트라 트리도 특성 중요도를 제공하며, 회귀 버전인 ExtraTreeRegressor 클래스도 존재한다.

In [8]:
et.fit(train_input, train_target)
print(et.feature_importances_)

[0.20183568 0.52242907 0.27573525]


# 그래디언트 부스팅

깊이가 얕은 결정 트리를 사용하여 이전 트리의 오차를 보완하는 방식으로 앙상블 하는 방법 (결정 트리를 연속적으로 추가하며 손실 함수를 최소화)
- 사이킷런의 GradientBoostingClassifier는 기본적으로 깊이가 3인 결정 트리를 100개 사용한다.
    - 따라서 과대적합에 강하고, 일반화 성능이 높다.
- 이름에서 알 수 있듯이 경사 하강법을 사용하여 트리를 앙상블에 추가한다.
    - 분류에서는 로지스틱 손실 함수를 사용하고,
    - 회귀에서는 평균 제곱 오차 함수를 사용한다.

경사 하강법
- 손실 함수(산)에서 가장 낮은 곳을 찾아 조금씩 천천히 내려가는 방법
    - 기존 경사 하강법: 모델의 가중치와 절편을 조금씩 바꾸기
    - 그래디언트 부스팅: 깊이가 얕은 결정 트리를 계속 추가하기

이제 이를 이용하여 와인 데이터셋의 교차 검증 점수를 확인해보자.

In [9]:
from sklearn.ensemble import GradientBoostingClassifier

gb = GradientBoostingClassifier(random_state=42)
scores = cross_validate(gb, train_input, train_target, return_train_score=True, n_jobs=-1)

print(np.mean(scores['train_score']), np.mean(scores['test_score']))

0.8881086892152563 0.8720430147331015


과대적합이 거의 해결되었다.

그래디언트 부스팅은 결정 트리 개수를 늘려도 과대적합에 매우 강하다.
- 학습률을 증가시키고 트리의 개수를 늘리면 조금 더 성능이 향상될 수 있다.

In [10]:
gb = GradientBoostingClassifier(n_estimators=500, learning_rate=0.2, random_state=42)
scores = cross_validate(gb, train_input, train_target, return_train_score=True, n_jobs=-1)

print(np.mean(scores['train_score']), np.mean(scores['test_score']))

0.9464595437171814 0.8780082549788999


결정 트리 개수를 5배 늘리고, 학습률을 2배로 늘렸다.

그럼에도 과대적합을 잘 억제하고 있다.

그래디언트 부스팅은 다른 트리 모델과 마찬가지로 특성 중요도를 제공한다.
- 단, 랜덤 포레스트보다 일부 특성(당도)에 더 집중한다.

In [11]:
gb.fit(train_input, train_target)
print(gb.feature_importances_)

[0.15872278 0.68010884 0.16116839]


그 외의 그래디언트 부스팅의 특징으로는,
- subsample 매개변수의 존재 (default: 1.0)
    - 이를 줄이면, 훈련 세트의 일부만을 이용할 수 있다.
    - 마치 경사 하강법 단계마다 일부 샘플을 랜덤하게 선택하는 확률적 경사 하강법이나 미니배치 경사 하강법 처럼.
- 일반적으로 랜덤 포레스트보다 조금 더 높은 성능을 얻을 수 있다.
    - 다만 병렬 훈련이 불가하기 때문에, 훈련 속도가 좀 느리고,
    - 그렇기에 n_jobs 매개변수가 없다.
- 회귀 버전으로 GradientBoostingRegressor이 존재한다.

## 히스토그램 기반 그래디언트 부스팅

정형 데이터를 다루는 머신러닝 알고리즘 중 가장 인기가 높은 알고리즘이다.

먼저 입력 특성을 256개의 구간으로 나눈다.
- 노드를 분할할 때 최적의 분할을 매우 빠르게 찾을 수 있다.
- 이 구간 중 하나를, 누락된 값을 위해서 사용한다.
    - 입력에 누락된 특성이 있더라도 이를 따로 전처리할 필요가 없다.

HistGradientBoostingClassifier 클래스
- 기본 매개변수에서 안정적인 성능을 얻을 수 있다.
- 트리의 개수를 지정하는 n_estimators 대신에, 부스팅 반복 횟수를 지정하는 max_iter를 사용한다.

In [13]:
# from sklearn.experimental import enable_hist_gradient_boosting
from sklearn.ensemble import HistGradientBoostingClassifier

hgb = HistGradientBoostingClassifier(random_state=42)
scores = cross_validate(hgb, train_input, train_target, return_train_score=True)

print(np.mean(scores['train_score']), np.mean(scores['test_score']))

0.9321723946453317 0.8801241948619236


과대적합을 잘 억제하면서 기존 그래디언트 부스팅보다 조금 더 성능이 높다.

특성 중요도를 확인해보자.

In [14]:
from sklearn.inspection import permutation_importance

hgb.fit(train_input, train_target)
result = permutation_importance(hgb, train_input, train_target, n_repeats=10, random_state=42, n_jobs=-1)

print(result.importances_mean)

[0.08876275 0.23438522 0.08027708]


permutation_importance() 함수
- 특성을 하나씩 랜덤하게 섞어서 모델의 성능이 변화하는지를 관찰함으로서, 어떤 특성이 중요한지를 계산한다.
    - 섞어서 성능이 떨어지면 --> 중요한 특성
    - 섞어서 성능 변화가 없거나 오히려 좋아지면 --> 안중요한 특성 / 해가 되는 특성
- 훈련 세트뿐만 아니라 테스트 세트에도 적용할 수 있고, 사이킷런에서 제공하는 추정기 모델에 모두 사용할 수 있다.
- n_repeats: 랜덤하게 섞을 횟수 (default: 5)
- 반환
    - 반복하여 얻은 특성 중요도 (importances)
    - 평균 (importances_mean)
    - 표준 편차 (importances_std)

이번에는 테스트 세트에 대해서도 특성 중요도를 계산해보자.

In [15]:
result = permutation_importance(hgb, test_input, test_target, n_repeats=10, random_state=42, n_jobs=-1)

print(result.importances_mean)

[0.05969231 0.20238462 0.049     ]


이제 최종적인 성능을 확인해보자

In [16]:
hgb.score(test_input, test_target)

0.8723076923076923

2절에서 측정한 랜덤 포레스트의 정확도(0.86) 보다는 높은 점수가 나왔다.

사이킷런 말고도 히스토그램 기반 그래디언트 부스팅 알고리즘을 구현한 라이브러리가 여럿 있다.

XGBoost
- 사이킷런의 cross_validate()와 함께 사용할 수도 있다.
- tree_method 매개변수로 'hist'로 지정하면 히스토그램 기반 그래디언트 부스팅을 사용할 수 있다.

In [17]:
from xgboost import XGBClassifier

xgb = XGBClassifier(tree_method='hist', random_state=42)
scores = cross_validate(xgb, train_input, train_target, return_train_score=True)

print(np.mean(scores['train_score']), np.mean(scores['test_score']))

0.8824322471423747 0.8726214185237284


또 다른 라이브러리는 MS의 LightGBM이 있다.

In [18]:
from lightgbm import LGBMClassifier

lgb = LGBMClassifier(random_state=42)
scores = cross_validate(lgb, train_input, train_target, return_train_score=True, n_jobs=-1)

print(np.mean(scores['train_score']), np.mean(scores['test_score']))

0.9338079582727165 0.8789710890649293


사이킷런의 HGB은 사실 LightGBM의 영향을 많이 받았다.

끝!